In [ ]:
pip install dash

In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import joblib
import json

In [18]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div(
    style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'padding': '20px'},  # This ensures all elements are stacked vertically
    children=[
        html.H1("Apartment Price Prediction"),

        # Square meters input
        html.Label("Square Meters"),
        dcc.Input(id='square_meters', type='number', min=10, max=500, value=50),

        # Floor input
        html.Label("Floor"),
        dcc.Input(id='floor', type='number', min=1, max=10, value=3),

        # Build year input
        html.Label("Build Year"),
        dcc.Input(id='build_year', type='number', min=1900, max=2025, value=2000),

        # Points of Interest input
        html.Label("Number of Points of Interest"),
        dcc.Input(id='poi_count', type='number', min=0, max=100, value=5),

        # Clinic distance input
        html.Label("Clinic Distance"),
        dcc.Input(id='clinic_distance', type='number', min=0, max=1000, value=50),

        # Kindergarten distance input
        html.Label("Kindergarten Distance"),
        dcc.Input(id='kindergarten_distance', type='number', min=0, max=1000, value=50),

        # College distance input
        html.Label("College Distance"),
        dcc.Input(id='college_distance', type='number', min=0, max=1000, value=50),

        # Pharmacy distance input
        html.Label("Pharmacy Distance"),
        dcc.Input(id='pharmacy_distance', type='number', min=0, max=1000, value=50),

        # Parking space input
        html.Label("Has Parking Space?"),
        dcc.RadioItems(
            id='has_parking_space',
            options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}],
            value='yes'
        ),

        # Elevator input
        html.Label("Has Elevator?"),
        dcc.RadioItems(
            id='has_elevator',
            options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}],
            value='yes'
        ),

        # Balcony input
        html.Label("Has Balcony?"),
        dcc.RadioItems(
            id='has_balcony',
            options=[{'label': 'Yes', 'value': 'yes'}, {'label': 'No', 'value': 'no'}],
            value='yes'
        ),

        # City selection input
        html.Label("City"),
        dcc.Dropdown(
            id='city',
            options=[
                {'label': 'Bialystok', 'value': 'bialystok'},
                {'label': 'Bydgoszcz', 'value': 'bydgoszcz'},
                {'label': 'Czestochowa', 'value': 'czestochowa'},
                {'label': 'Gdansk', 'value': 'gdansk'},
                {'label': 'Gdynia', 'value': 'gdynia'},
                {'label': 'Katowice', 'value': 'katowice'},
                {'label': 'Krakow', 'value': 'krakow'},
                {'label': 'Lodz', 'value': 'lodz'},
                {'label': 'Lublin', 'value': 'lublin'},
                {'label': 'Poznan', 'value': 'poznan'},
                {'label': 'Radom', 'value': 'radom'},
                {'label': 'Rzeszow', 'value': 'rzeszow'},
                {'label': 'Szczecin', 'value': 'szczecin'},
                {'label': 'Warszawa', 'value': 'warszawa'},
                {'label': 'Wroclaw', 'value': 'wroclaw'}
            ],
            value='warszawa',
            style={'width': '200px'}
        ),

        # Property condition input
        html.Label("Condition"),
        dcc.Dropdown(
            id='condition',
            options=[
                {'label': 'Standard', 'value': 'standard'},
                {'label': 'Premium', 'value': 'premium'},
                {'label': 'Low', 'value': 'low'}
            ],
            value='standard',
            style={'width': '200px'}
        ),

        # Button to trigger prediction
        html.Button('Predict', id='predict-button', n_clicks=0),

        # Output: Predicted Price per Square Meter
        html.Div(id='prediction-output')
    ]
)


In [19]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>